<a href="https://colab.research.google.com/github/ebaenamar/aiart/blob/main/Finetune_SDXL_Replicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use

To run and modify the notebook, in the top left go to file -> make a copy in Drive.

Useful shortcuts:
- Shift + enter: runs a cell

Additional Resources:

More in depth fine tuning explanation [here](https://civitai.com/articles/4/make-your-own-loras-easy-and-free)

In [ ]:
!pip install replicate

Test Run

# Finetuning a text to image model

The first and most important thing to care about when training a custom image generation model, is the data. If you have a bad dataset that you are trianing on, it does not matter what model or how much compute you throw at the problem, your output model will still not perform the way that you want it to.

For image generation, we dont actually need a lot of data to add a new concept or style to the model, as little as 5 images will do, although more is always better, usually datasets are between 20-1000 miages. When selecting images here's what you need to keep in mind:

- Avoid low quality images, i.e. blurry or low (<256 px) resolution
- Avoid images with weird aspect ratios (anything more than 2:1, ie 1024x512px)
- Dont worry about getting 4k or super high resolution images, they will be downscaled to ~1024px per side when training

When training a model, you will typically either be training the model to understand a person, or new style. Because of this, you will usually include a trigger word that lets the model know you are trying to evoke that concept. That way the model will keep its previous understanding of concepts while also having a new one added to it. Because we dont want to overwrite existing concepts, the trigger word will be a specific person's name, or a "custom" word, i.e. "Andrew Mead" or "tr1gg3r w0rd".

In [ ]:
#@title Setup Replicate

#@markdown To get your Replicate API key, go to [Replicate](https://replicate.com/signin?next=/docs) and register. You then find your api key on the [API tokens page](https://replicate.com/account/api-tokens), which you can then paste here.

import os
import replicate
# YOUR REPLICATE API KEY
replicate_api_key = "" #@param {type: 'string'}

os.environ["REPLICATE_API_TOKEN"] = replicate_api_key

Test Run

In [ ]:
output = replicate.run(
    "stability-ai/sdxl:7762fd07cf82c948538e41f63f77d685e02b063e37e496e96eefd46c929f9bdc",
    input={
        "width": 768,
        "height": 768,
        "prompt": "An astronaut riding a rainbow unicorn, cinematic, dramatic",
        "refine": "expert_ensemble_refiner",
        "scheduler": "K_EULER",
        "lora_scale": 0.6,
        "num_outputs": 1,
        "guidance_scale": 7.5,
        "apply_watermark": False,
        "high_noise_frac": 0.8,
        "negative_prompt": "",
        "prompt_strength": 0.8,
        "num_inference_steps": 25
    }
)
print(output)

NameError: name 'replicate' is not defined

In [ ]:
#@title Create the model repository

#@markdown Here we are setting up the repository in replicate where the model will go once we have trained it

import replicate
from replicate.exceptions import ReplicateError

#@markdown You can see your username on replicate in the top left corner.
replicate_username = "" #@param {type: 'string'}
#@markdown Name of your fintuned model
finetuned_mode_name = "" #@param {type: 'string'}

try:
  model = replicate.models.create(
      owner=replicate_username,
      name=finetuned_mode_name,
      visibility="public",  # or "private" if you prefer
      hardware="gpu-t4",  # Replicate will override this for fine-tuned models
      description="A fine-tuned sdxl model"
  )
  print(f"Model created: {model.name}")
except ReplicateError as e:
  if "already exists" in e.detail:
    print("Model already exists, loading it.")
    model = replicate.models.get(f"{replicate_username}/{finetuned_mode_name}")
  else:
    raise e

print(f"Model URL: https://replicate.com/{model.owner}/{model.name}")

In [ ]:
#@title Train the model

#@markdown The dataset needs to be a zip folder, with
dataset_url = "" #@param {type: 'string'}
trigger_word = "" #@param {type: 'string'}
steps = None #@param {type: 'number'}

training = replicate.trainings.create(
    version="stability-ai/sdxl:7762fd07cf82c948538e41f63f77d685e02b063e37e496e96eefd46c929f9bdc",
    input={
        "input_images": dataset_url,
        "steps": steps,
        "token_string": trigger_word,
        "is_lora": "true"
    },
    destination=f"{model.owner}/{model.name}",
)

print(f"Training started: {training.status}")
print(f"Training URL: https://replicate.com/p/{training.id}")

In [ ]:
#@title Use your model
prompt = "" #@param {type: 'string'}

latest_version = model.versions.list()[0]

output = replicate.predictions.create(
    version=latest_version,
    input={"prompt": prompt}
)

print(output)

In [ ]:
output